# University of Waterloo Student Housing and Venue Capstone Project

By: Jacob Rintjema

Date: November 2020

---

# Introduction & Description of the Background

The University of Waterloo (commonly referred to as Waterloo, UW, or UWaterloo) is a public research university with a main campus in Waterloo, Ontario, Canada. The main campus is on 404 hectares (998 acres) of land adjacent to "Uptown" Waterloo and Waterloo Park. The university also operates three satellite campuses and four affiliated university colleges. The university offers academic programs administered by six faculties and thirteen faculty-based schools. Waterloo operates the largest post-secondary co-operative education program in the world, with over 20,000 undergraduate students enrolled in the university's co-op program. The university is a co-educational institution, with approximately 41,000 undergraduate and 6,900 postgraduate students enrolled there in 2019.

The focus of this project is to provide housing rental and purchashing options for students on and off campus. For incoming students, (especially international level), there is always stress and difficulties involved finding housing. This service will provide affordable options that tailor towards housing near the university campus. Students will get situated easier and will be able to focus more on their studies instead. The service will leverage Four Square API data to support this.

# Data Description

The project will provide:
 - Housing data from the Realtor - Canadian Real Estate API, Ex. Housing options within a 2 km range from campus.
 - Nearby venue options using the Four Square API, Ex. Shopping centres, restaurants and bars nearby.
 - Location of housing using GeoPy, Ex: Longitude and latitude coordinates of housing.
 
Students will be able to view housing locations as well as close by venues that are in range of the University of Waterloo campus.

Data Needed:
 - Realtor API Data
 - GeoPy Location Data
 - Four Square API Data
 - Waterloo Geo Boundaries

# Resources

 - [Realtor API](https://rapidapi.com/apidojo/api/realtor-canadian-real-estate?endpoint=apiendpoint_b55bd49d-f8a8-4728-a397-c74a56d9d921)
 - [Four Square API](https://developer.foursquare.com)
 - [Waterloo Geo Boundaries](https://data.waterloo.ca/datasets/RMW::regional-boundaries-1/geoservice?geometry=-81.519%2C43.295%2C-79.541%2C43.644)